In [16]:
print(__doc__)

import numpy as np
import scipy as sp
import pandas as pd
from subprocess import check_output
from sklearn.svm import SVR

import matplotlib.pyplot as plt
%matplotlib inline

import csv

Automatically created module for IPython interactive environment


In [17]:

#read the csv into a dataframe
df = pd.read_csv('nseComp.1.csv', parse_dates = ['Date'] )
#print the contents of the dataframe

In [18]:
df2 = df.drop(['Unnamed: 0'], axis = 1)
df2.describe()

,Open,High,Low,Close,Volume
count,5.803312e+06,5.803312e+06,5.803312e+06,5.803312e+06,5.803312e+06
mean,1.084660e+03,1.085290e+03,1.084010e+03,1.084647e+03,4.030938e+03
std,2.754575e+03,2.756003e+03,2.753105e+03,2.754518e+03,2.431125e+04
min,2.300000e+00,2.300000e+00,2.300000e+00,2.300000e+00,1.000000e+00
25%,1.761500e+02,1.763000e+02,1.760000e+02,1.761500e+02,8.600000e+01
50%,4.614000e+02,4.617000e+02,4.610000e+02,4.614000e+02,4.950000e+02
75%,1.098650e+03,1.099400e+03,1.098000e+03,1.098650e+03,2.484000e+03
max,2.797950e+04,2.799000e+04,2.795000e+04,2.797005e+04,1.305972e+07


In [19]:
#remove null value (preprocessing)
df2 = df2.dropna()   
df2.count() 

Code      5803312
Date      5803312
Time      5803312
Open      5803312
High      5803312
Low       5803312
Close     5803312
Volume    5803312
dtype: int64

In [20]:
#set index for locating companies
df3=df2.set_index("Code")
df3.dtypes

Date      datetime64[ns]
Time              object
Open             float64
High             float64
Low              float64
Close            float64
Volume           float64
dtype: object

In [47]:
uniqueVals

NameError: name 'uniqueVals' is not defined

In [49]:
# creates an array of company names
uniqueVals = df2["Code"].unique() 
# appends the results of all companies
grouped_df=pd.DataFrame()  # empty dataframe
for i in uniqueVals:
    # store DataFrame in list
    grouped_df=grouped_df.append((df3.loc[i,:]).groupby(['Code','Date']).mean())
#grouped_df = append_df.reset_index()
uniqueVals


array(['3IINFOTECH', '8KMILES', 'ABAN', 'ABB', 'ABBOTINDIA', 'ABIRLANUVO',
       'ACC', 'ADANIENT', 'ADANIPORTS', 'ADANIPOWER', 'AHMEDFORGE',
       'AIAENG', 'AIL', 'AJANTPHARM', 'AKZOINDIA', 'ALBK', 'ALEMBICLTD',
       'ALLCARGO', 'ALOKTEXT', 'AMARAJABAT', 'AMBUJACEM', 'AMTEKAUTO',
       'AMTEKINDIA', 'ANANTRAJ', 'ANDHRABANK', 'APLLTD', 'APOLLOHOSP',
       'APOLLOTYRE', 'ARCOTECH', 'ARVIND', 'ARVINDREM', 'ASHOKA',
       'ASHOKLEY', 'ASIANPAINT', 'ASTRAL', 'ASTRAMICRO', 'ATFL',
       'ATULAUTO', 'AURIONPRO', 'AUROPHARMA', 'AXISBANK', 'AXISCADES',
       'BAJAJCORP', 'BAJAJELEC', 'BAJAJFINSV', 'BAJAJHIND', 'BAJAJHLDNG',
       'BAJFINANCE', 'BALKRISIND', 'BALMLAWRIE', 'BALRAMCHIN',
       'BANKBARODA', 'BANKINDIA', 'BATAINDIA', 'BAYERCROP', 'BBL', 'BBTC',
       'BEL', 'BEML', 'BERGEPAINT', 'BFUTILITIE', 'BGRENERGY',
       'BHARATFORG', 'BHARTIARTL', 'BHEL', 'BIOCON', 'BLUEDART',
       'BOMDYEING', 'BOSCHLTD', 'BPCL', 'BPL', 'BRITANNIA', 'CADILAHC',
       'CAIRN', 'CANBK', 'CA

In [24]:
#grouped_CHAMBLFERT= (grouped_df.loc["CHAMBLFERT",:]).drop(['Code', index = False])
grouped_infotech = (grouped_df.loc["3IINFOTECH",:]).reset_index()

In [26]:
grouped_infotech.to_csv('infotech.csv', encoding='utf-8', index=False) #converting into csv

In [42]:
#plt.switch_backend('newbackend') 
import csv
dates = []
prices = []

def get_data(filename):
	with open(filename, 'r') as csvfile:
		csvFileReader = csv.reader(csvfile)
		next(csvFileReader)	# skipping column names
		for row in csvFileReader:
			dates.append(int(row[0].split('-')[2]))
			prices.append(float(row[1]))
	return

get_data('infotech.csv')


In [43]:
def create_datasets(dates,prices):
    train_size=int(0.90*len(dates))
    print train_size
    TrainX,TrainY=[],[]
    TestX,TestY=[],[]
    cntr=0
    for date in dates:
        if cntr<train_size:
            TrainX.append(date)
        else:
            TestX.append(date)
        cntr = cntr+1
    cntr=0
    for price in prices:
        if cntr<train_size:
            TrainY.append(price)
        else:
            TestY.append(price)
        cntr = cntr+1
        
    return TrainX,TrainY,TestX,TestY

In [46]:
 TrainX,TrainY,TestX,TestY = create_datasets(dates, prices)
 print TestX

142
[8, 9, 10, 11, 14, 15, 16, 18, 21, 22, 23, 24, 28, 29, 30, 1]


In [31]:
def predict_price_test(dates, prices, x):
	dates = np.reshape(dates,(len(dates), 1)) # converting to matrix of n X 1

	svr_rbf = SVR(kernel= 'rbf', C= 1e3, gamma= 0.1) # defining the support vector regression models
	svr_rbf.fit(dates, prices) # fitting the data points in the models
	#print("jhjgj ",svr_rbf.predict(-1)[0])
    #plt.plot(dates, svr_rbf.predict(dates), color= 'red', label= 'RBF model')
    
	return svr_rbf.predict(x)[0]

In [33]:
predicted_price = predict_price_test(dates, prices, 29)  
predicted_price

6.936689506405

In [36]:
predicted_prices = list()
for i in range(30):
    predicted_prices.append(predict_price_test(dates, prices, i+1))
    #predicted_prices = predicted_prices.append(val)
   # predicted_prices = predicted_prices.append(predict_price_test(dates, prices, i))
#plt.plot(predicted_prices)

In [37]:
predicted_prices

[6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405,
 6.936689506405]

In [ ]:
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
error = math.sqrt(mean_squared_error(prices, predictions))
print('Test MSE: %.3f' % error)